In [120]:
### Import libraries

# Fix randomness and hide warnings
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

import numpy as np
np.random.seed(seed)

import logging

import random
random.seed(seed)

# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler

2.13.0


In [121]:
#Array che contiene le categorie di ogni time series. Dim 48000x1
categories = np.load("/kaggle/input/dataset-homework2/categories.npy")
#Array che contiene i valori delle time series. Dim 48000x2776
training_data = np.load("/kaggle/input/dataset-homework2/training_data.npy")
#Array che contiene gli indici che descrivono gli estremi degli intervalli validi delle time series. Dim 48000x2
valid_periods = np.load("/kaggle/input/dataset-homework2/valid_periods.npy")

categories.shape, training_data.shape, valid_periods.shape



((48000,), (48000, 2776), (48000, 2))

In [122]:
def clean_data(training_data, garbage_threshold):
    cleaned_data = []
    cleaned_categories = []
    cleaned_valid_periods = []
    for i in range(training_data.shape[0]):
        if valid_periods[i][1] - valid_periods[i][0] >= garbage_threshold:
            cleaned_data.append(training_data[i])
            cleaned_categories.append(categories[i])
            cleaned_valid_periods.append(valid_periods[i])
    return np.array(cleaned_data), np.array(cleaned_categories), np.array(cleaned_valid_periods)

In [123]:
def split_for_categories(training_data, categories):
    all_samples_list = []
    for i in np.unique(categories):
        mask = np.where(categories == i, True, False)

        samples_for_category = training_data[mask]
        all_samples_list.append(samples_for_category)
    return np.array(all_samples_list[0]), np.array(all_samples_list[1]), np.array(all_samples_list[2]), np.array(all_samples_list[3]), np.array(all_samples_list[4]), np.array(all_samples_list[5])

In [124]:
cleaned_data, cleaned_categories, cleaned_valid_periods = clean_data(training_data, 600)

cleaned_categories.shape, cleaned_data.shape, cleaned_valid_periods.shape

((609,), (609, 2776), (609, 2))

In [125]:
data_A, data_B, data_C, data_D, data_E, data_F = split_for_categories(cleaned_data, cleaned_categories)

data_A.shape, data_B.shape, data_C.shape, data_D.shape, data_E.shape, data_F.shape

((82, 2776), (106, 2776), (190, 2776), (160, 2776), (64, 2776), (7, 2776))

In [126]:
def train_test_split(data, train_ratio = 0.8):
    data = data
    
    rows_to_select = int(len(data) * train_ratio)
    training_set = data[:rows_to_select, :]
    test_set = data[rows_to_select:, :]
    
    return training_set, test_set

In [127]:
train_validation_C, test_C = train_test_split(data_C)
train_C, validation_C = train_test_split(train_validation_C)
len(train_C), len(validation_C), len(test_C)

(121, 31, 38)

In [128]:
def build_sequences(df, window, stride, telescope):
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy()
    padding_check = df.size%window

    #print(temp_df.size)

    if(padding_check != 0):
        # Compute padding length
        padding_len = window - df.size%window
        padding = np.zeros((padding_len), dtype='float32')
        temp_df = np.concatenate((padding,df))
        assert temp_df.size % window == 0

    #print(temp_df.size)
    for idx in np.arange(0,temp_df.size-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_df[idx+window:idx+window+telescope])

    return dataset, labels

In [129]:
#data is a pandas series containing 48000 lists (either training, validation or test)
def THE_SEQUENCER(data,g,w,s,t):
    dataset = []
    labels = []
    
    cleaned_data, cleaned_categories, cleaned_valid_periods = clean_data(data, g)

    for i in range(len(cleaned_data)):
        time_series = cleaned_data[i][cleaned_valid_periods[i][0]:cleaned_valid_periods[i][1]]
        dset, labs = build_sequences(time_series, w, s, t)
        if len(dset) == 0:
            continue
        dataset.append(dset)
        labels.append(labs)

    return dataset, labels

In [130]:
train_sequences_C, train_labels_C = THE_SEQUENCER(train_C,400, 200, 50, 9)
validation_sequences_C, validation_labels_C = THE_SEQUENCER(train_C,400, 200, 50, 9)
test_sequences_C, test_labels_C = THE_SEQUENCER(data_C,400, 200, 50, 9)

In [131]:
def build_CONV_LSTM_model(input_shape, output_shape, num_neurons):
    # Ensure the input time steps are at least as many as the output time steps
    assert input_shape[0] >= output_shape[0], "For this exercise we want input time steps to be >= of output time steps"
    # Define the input layer with the specified shape
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')
    # Add a Bidirectional LSTM layer with 64 units

    x = tfkl.Bidirectional(tfkl.LSTM(num_neurons, return_sequences=True, name='lstm'), name='bidirectional_lstm')(input_layer)
    
    # Add a 1D Convolution layer with 128 filters and a kernel size of 3
    x = tfkl.Conv1D(128, 3, padding='same', activation='relu', name='conv')(x)
    # Add a final Convolution layer to match the desired output shape
    output_layer = tfkl.Conv1D(output_shape[1], 3, padding='same', name='output_layer')(x)
    # Calculate the size to crop from the output to match the output shape
    crop_size = output_layer.shape[1] - output_shape[0]
    # Crop the output to the desired length
    output_layer = tfkl.Cropping1D((0, crop_size), name='cropping')(output_layer)
    # Construct the model by connecting input and output layers
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name='CONV_LSTM_model')
    # Compile the model with Mean Squared Error loss and Adam optimizer
    model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam())
    return model

In [132]:
def to_arr(array, label):
    X_train = []
    y_train = []
    for i in range(len(array)):
        for j in range(len(array[i])):
            X_train.append(array[i][j])
            y_train.append(label[i][j])
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    return X_train, y_train

In [133]:
X_valid = []
y_valid = []
for i in range(len(validation_sequences_C)):
    for j in range(len(validation_sequences_C[i])):
        X_valid.append(validation_sequences_C[i][j])
        y_valid.append(validation_labels_C[i][j])
X_valid = np.array(X_valid)
y_valid = np.array(y_valid)

In [134]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [135]:
import optuna
input_shape = (200,1)
output_shape = (9,1)

def objective_function(optuna_trial):
    
    num_neurons = optuna_trial.suggest_int("num_neurons", 10, 128)
    batch_size = optuna_trial.suggest_int("batch_size", 32, 128)
    garbage = optuna_trial.suggest_int("garbage", 210, 840)
    stride = optuna_trial.suggest_categorical("stride", [1,2,5,10,20,25,40,50])
    
    
    
    train_sequences_C, train_labels_C = THE_SEQUENCER(train_C,garbage, 200, stride, 9)
    validation_sequences_C, validation_labels_C = THE_SEQUENCER(train_C,garbage, 200, stride, 9)
    
    
    X_train, y_train = to_arr(train_sequences_C, train_labels_C) 
    X_valid, y_valid = to_arr(validation_sequences_C, validation_labels_C) 

    # Generate our trial model.

    

    print(f"Current trial parameters: num_neurons={num_neurons}, batch_size={batch_size}, garbage = {garbage}, stride = {stride}")

    model = build_CONV_LSTM_model(input_shape, output_shape, num_neurons)
    
    # Define callbacks for early stopping and learning rate reduction
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', mode='min', patience=12, restore_best_weights=True,min_delta=0.00001
    )
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', mode='min', patience=10, factor=0.1, min_lr=1e-5
    )

    # Fit the model on the training data.
    # The TFKerasPruningCallback checks for pruning condition every epoch.
    model.fit(
    x = X_train,
    y = y_train,
    batch_size = batch_size, 
    epochs = 300,
    validation_data=(X_valid, y_valid),
    callbacks = [early_stopping, reduce_lr]
    )

    # Evaluate the model accuracy on the validation set.
    return model.evaluate(X_valid, y_valid, verbose=0)

In [136]:
optuna_study = optuna.create_study(direction="minimize")

[I 2023-12-15 15:31:49,610] A new study created in memory with name: no-name-d42fafe6-96cb-4847-8176-f690ca9685e8


In [ ]:
optuna_study = optuna.create_study(direction="minimize")
optuna_study.optimize(objective_function,
                    n_trials = 50)

[I 2023-12-15 15:31:49,621] A new study created in memory with name: no-name-465a4064-d38d-4f60-8749-e8bf829d9adb


Current trial parameters: num_neurons=100, batch_size=47, garbage = 746, stride = 2
Epoch 1/300
76/76 [==============================] - 59s 45ms/step - loss: 0.0456 - val_loss: 0.0381 - lr: 0.0010
Epoch 2/300
76/76 [==============================] - 3s 34ms/step - loss: 0.0359 - val_loss: 0.0277 - lr: 0.0010
Epoch 3/300
76/76 [==============================] - 3s 34ms/step - loss: 0.0254 - val_loss: 0.0352 - lr: 0.0010
Epoch 4/300
76/76 [==============================] - 3s 34ms/step - loss: 0.0395 - val_loss: 0.0355 - lr: 0.0010
Epoch 5/300
76/76 [==============================] - 3s 34ms/step - loss: 0.0359 - val_loss: 0.0361 - lr: 0.0010
Epoch 6/300
76/76 [==============================] - 3s 34ms/step - loss: 0.0353 - val_loss: 0.0357 - lr: 0.0010
Epoch 7/300
76/76 [==============================] - 3s 34ms/step - loss: 0.0344 - val_loss: 0.0318 - lr: 0.0010
Epoch 8/300
76/76 [==============================] - 3s 34ms/step - loss: 0.0383 - val_loss: 0.0349 - lr: 0.0010
Epoch 9/300

[I 2023-12-15 15:33:23,544] Trial 0 finished with value: 0.02769436314702034 and parameters: {'num_neurons': 100, 'batch_size': 47, 'garbage': 746, 'stride': 2}. Best is trial 0 with value: 0.02769436314702034.


Current trial parameters: num_neurons=32, batch_size=78, garbage = 790, stride = 20
Epoch 1/300
5/5 [==============================] - 5s 226ms/step - loss: 0.1566 - val_loss: 0.0743 - lr: 0.0010
Epoch 2/300
5/5 [==============================] - 0s 36ms/step - loss: 0.0643 - val_loss: 0.0659 - lr: 0.0010
Epoch 3/300
5/5 [==============================] - 0s 33ms/step - loss: 0.0644 - val_loss: 0.0540 - lr: 0.0010
Epoch 4/300
5/5 [==============================] - 0s 32ms/step - loss: 0.0493 - val_loss: 0.0476 - lr: 0.0010
Epoch 5/300
5/5 [==============================] - 0s 32ms/step - loss: 0.0472 - val_loss: 0.0449 - lr: 0.0010
Epoch 6/300
5/5 [==============================] - 0s 32ms/step - loss: 0.0431 - val_loss: 0.0421 - lr: 0.0010
Epoch 7/300
5/5 [==============================] - 0s 32ms/step - loss: 0.0429 - val_loss: 0.0415 - lr: 0.0010
Epoch 8/300
5/5 [==============================] - 0s 32ms/step - loss: 0.0410 - val_loss: 0.0405 - lr: 0.0010
Epoch 9/300
5/5 [==========

[I 2023-12-15 15:33:47,828] Trial 1 finished with value: 0.019529910758137703 and parameters: {'num_neurons': 32, 'batch_size': 78, 'garbage': 790, 'stride': 20}. Best is trial 1 with value: 0.019529910758137703.


...

Current trial parameters: num_neurons=22, batch_size=84, garbage = 560, stride = 2
Epoch 1/300
109/109 [==============================] - 7s 32ms/step - loss: 0.0527 - val_loss: 0.0372 - lr: 0.0010
Epoch 2/300
109/109 [==============================] - 3s 24ms/step - loss: 0.0378 - val_loss: 0.0352 - lr: 0.0010
Epoch 3/300
109/109 [==============================] - 3s 24ms/step - loss: 0.0334 - val_loss: 0.0280 - lr: 0.0010
Epoch 4/300
109/109 [==============================] - 3s 24ms/step - loss: 0.0251 - val_loss: 0.0205 - lr: 0.0010
Epoch 5/300
109/109 [==============================] - 3s 24ms/step - loss: 0.0168 - val_loss: 0.0141 - lr: 0.0010
Epoch 6/300
109/109 [==============================] - 3s 24ms/step - loss: 0.0122 - val_loss: 0.0110 - lr: 0.0010
Epoch 7/300
109/109 [==============================] - 3s 24ms/step - loss: 0.0096 - val_loss: 0.0124 - lr: 0.0010
Epoch 8/300
109/109 [==============================] - 3s 24ms/step - loss: 0.0096 - val_loss: 0.0085 - lr: 0.00

[I 2023-12-15 15:41:31,800] Trial 9 finished with value: 0.002435969654470682 and parameters: {'num_neurons': 22, 'batch_size': 84, 'garbage': 560, 'stride': 2}. Best is trial 9 with value: 0.002435969654470682.


Current trial parameters: num_neurons=13, batch_size=97, garbage = 412, stride = 2
Epoch 1/300
124/124 [==============================] - 7s 29ms/step - loss: 0.0588 - val_loss: 0.0434 - lr: 0.0010
Epoch 2/300
124/124 [==============================] - 3s 23ms/step - loss: 0.0427 - val_loss: 0.0421 - lr: 0.0010
Epoch 3/300
124/124 [==============================] - 3s 23ms/step - loss: 0.0417 - val_loss: 0.0412 - lr: 0.0010
Epoch 4/300
124/124 [==============================] - 3s 23ms/step - loss: 0.0411 - val_loss: 0.0418 - lr: 0.0010
Epoch 5/300
124/124 [==============================] - 3s 23ms/step - loss: 0.0399 - val_loss: 0.0384 - lr: 0.0010
Epoch 6/300
124/124 [==============================] - 3s 23ms/step - loss: 0.0340 - val_loss: 0.0298 - lr: 0.0010
Epoch 7/300
124/124 [==============================] - 3s 23ms/step - loss: 0.0243 - val_loss: 0.0290 - lr: 0.0010
Epoch 8/300
124/124 [==============================] - 3s 23ms/step - loss: 0.0291 - val_loss: 0.0302 - lr: 0.00

[I 2023-12-15 15:43:00,418] Trial 10 finished with value: 0.014970776624977589 and parameters: {'num_neurons': 13, 'batch_size': 97, 'garbage': 412, 'stride': 2}. Best is trial 9 with value: 0.002435969654470682.


Current trial parameters: num_neurons=82, batch_size=63, garbage = 222, stride = 1
Epoch 1/300
454/473 [===========================>..] - ETA: 0s - loss: 0.0527

In [ ]:
print(optuna_study.best_trial.params)
print(optuna_study.best_trial.value)